# JUICE RPWI HF SID21 (PSSR1 rich): L1a QL -- 2025/10/14

# Import lib

In [ ]:
import copy
import datetime
# import math
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import sys

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf
import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cal_lib    as juice_cal
# import juice_cdf_lib    as juice_cdf
import juice_math_lib   as juice_math
# import juice_sid21_data as juice_data
import juice_sid21_lib  as juice_sid21
import juice_spec_lib   as juice_spec

# Setting and Read CDF file: set by User

In [ ]:
# *** Dump ***
dump_mode = 0                           # 0: no-dump  1:plot dump
# *** CAL ***
unit_mode = 0                           # [Power]     0: raw     1: V＠ADC     2: V@HF    3: V@RWI  4: V/m
band_mode = 0                           # [Power]     0: sum     1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all
# *** Unit ***
f_mode    = 1                           # [Frequency] 0: linear  1: log
time_mode = 1                           # [Time]      0: Epoch   1: data number
gap_mode  = 1                           # [gap]       0: n/a     1: gap
# *** Power range ***
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
N_ch0     = 4                           # [channel]   2: 2-ch    3: 3-ch    >3: any
comp_mode = 4                           # [Complex]   0: Poweer  1: Matrix  >3: any

In [ ]:
E_min = '2024-08-19 20:30:00';  t_min0 = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S")
E_max = '2024-08-19 20:40:00';  t_max0 = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0

# get CDF data

In [ ]:
# date='20240909'
# date='20250331'; ver = 'V02'
date='0'; ver = 'V02'
data_dir, data_name_list = juice_sid21.datalist(date, ver)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)
for i in range(num_list):
    data_name = data_name_list[i];  cdf_file = data_dir + data_name;  print(i, cdf_file)
    # RPWI_FSW_version = juice_cdf._RPWI_FSW_version(cdf_file)
    cdf = pycdf.CDF(cdf_file);      data1 = juice_sid21.hf_sid21_read(cdf) #, RPWI_FSW_version)
    if i==0:
        data = data1;                                  print(data.EuEu.shape)
    else:
        data = juice_sid21.hf_sid21_add(data, data1);  print(data.EuEu.shape)
data_name = os.path.split(data_name)[1];               print("data name:", data_name)

In [ ]:
data = juice_sid21.hf_sid21_shaping(data, cal_mode, N_ch0, comp_mode)

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.EuEu.shape[0];   n_freq0 = data.EuEu.shape[1]
f_min0  = data.frequency[0][0]; f_max0  = (max(np.ravel(data.frequency)))

if data.n_time>1: 
    date3 = data.epoch[1];  date3 = date3.strftime('%Y/%m/%d %R:%S')
    print("       date and time:", str_date, "(interval:", data.epoch[1] - data.epoch[0], date3, ")")
else:
    print("       date and time:", str_date)
print("           data size:", data.EuEu.shape, "  [ Num-samples:", n_time0, "  Num-Frequency:", n_freq0, "]")
print("           Frequency:", f_min0,  "-", f_max0, "kHz")
print("f, width, step (kHz):", f_min0,  "-", f_max0, data.freq_width[0][0], data.freq_step[0][0])

if N_ch0     > 3:   N_ch0     = (data.ch_selected[0] & 0b1) + ((data.ch_selected[0] & 0b10) >> 1) + ((data.ch_selected[0] & 0b100) >> 2)
if comp_mode > 3:   comp_mode = data.complex[0]
print("                  Ch:", data.ch_selected[0])
#
N_frequency = data.frequency.shape[1]
if (comp_mode == 0):
    print("             complex:", comp_mode, " (power 2-ch) <Table 6.5.2>")
    N_component = N_ch0;          print("       ==> Component:", N_component)
else:
    print("             complex:", comp_mode, " (nominal spectral matrix 2-ch) <Table 6.5.3>")
    N_component = N_ch0 * N_ch0;  print("       ==> Component:", N_component)


# Spectrum Data

In [ ]:
T_HF  = 25;  T_RWI = 25     # No Temperature data in TLM
print("Temperature: ", 'HF - {:.1f}'.format(T_HF), '  RWI - {:.1f}'.format(T_RWI), ' <No Temp data in SID-21>')

In [ ]:
spec = copy.deepcopy(data)
spec = juice_cal.spec_cal(spec,  21, unit_mode, band_mode, T_HF, T_RWI)     # CAL
spec.EE = spec.EuEu + spec.EvEv + spec.EwEw

Epoch_1d  = spec.epoch.tolist();      n_time1 = spec.EuEu.shape[0];  num_1d = np.arange(n_time1)
freq_1d   = spec.freq  [n_time1//2];  n_freq1 = freq_1d.shape[0];    f_min0 = freq_1d[0];  f_max0  = freq_1d[-1]
freq_w_1d = spec.freq_w[n_time1//2]

# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(n_time1-1):
        if spec.scet[i+1]-spec.scet[i] > 60:
            print("[gap]", i, spec.epoch[i], i+1, spec.epoch[i+1])
            juice_sid21.spec_nan(spec, i);  juice_sid21.spec_nan(spec, i+1)

In [ ]:
spec = juice_spec.hf_getspec_angle    (spec, 21)   # Coherency & ange
spec = juice_spec.hf_getspec_stokes   (spec, 21)   # Stokes and Polarization Parameters 

In [ ]:
p_max0 = p_raw_max + spec.cf/10;  p_min0 = p_raw_min + spec.cf/10
if   data.U_selected[n_time1//2] == 1 and data.V_selected[n_time1//2] == 1:
    p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EuEu), np.nanmax(spec.EvEv)] ) )*5)/5+.5;  p_min0 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EuEu), np.nanmin(spec.EvEv)] ) )*5)/5-.5
elif data.U_selected[n_time1//2] == 1 and data.W_selected[n_time1//2] == 1:
    p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EvEv), np.nanmax(spec.EwEw)] ) )*5)/5+.5;  p_min0 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EvEv), np.nanmin(spec.EwEw)] ) )*5)/5-.5
else:
    p_max0 = np.ceil(np.log10( np.nanmax( [np.nanmax(spec.EuEu), np.nanmax(spec.EwEw)] ) )*5)/5+.5;  p_min0 = np.ceil(np.log10( np.nanmin( [np.nanmin(spec.EuEu), np.nanmin(spec.EwEw)] ) )*5)/5-.5
p_max  = p_max0;                  p_min  = p_min0
print("conversion factor:", '{:.1f}'.format(spec.cf), "    MAX-min:", '{:.1f}'.format(p_max), '({:.1f})'.format(p_max0), '{:.1f}'.format(p_min), '({:.1f})'.format(p_min0), "   str_unit:", spec.str_unit)

### Peak

In [ ]:
# print("N_samp:", data.N_samp[0], "   decimation:", juice_cdf._sample_rate(data.decimation[0]))
if data.U_selected[n_time1//2] == 1:
    peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E)
    print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
if data.V_selected[n_time1//2] == 1:
    peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E)
    print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
if data.W_selected[n_time1//2] == 1:
    peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E)
    print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, ' {:.2e}'.format(peak_E[peak_f]**.5), '  |  {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])), spec.str_unit, ' {:.2e}'.format( (peak_E[peak_f-2]+peak_E[peak_f-1]+peak_E[peak_f]+peak_E[peak_f+1]+peak_E[peak_f+2])**.5))
if unit_mode==0:
    if data.U_selected[n_time1//2] == 1:  peak_E = np.ravel(spec.EuEu[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EuEu:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    if data.V_selected[n_time1//2] == 1:  peak_E = np.ravel(spec.EvEv[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EvEv:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))
    if data.W_selected[n_time1//2] == 1:  peak_E = np.ravel(spec.EwEw[:][:]); peak_f = np.nanargmax(peak_E);  print("Peak EwEw:", '{:10.0f}'.format(peak_E[peak_f]), '{:x}'.format(np.int64(peak_E[peak_f])))

In [ ]:
n_sweep = n_time1//2;  peak_E = np.ravel(spec.EuEu[n_sweep]);  peak_f = np.nanargmax(peak_E); 
print(f"[SWEEP - {n_sweep}]  Peak at", '{:.1f}'.format(freq_1d[peak_f]), "kHz", peak_f)
print("     EuEu:", '{:+.2e}'.format(np.ravel(spec.EuEu   [n_sweep])[peak_f]), spec.str_unit, "     EvEv:", '{:+.2e}'.format(np.ravel(spec.EvEv   [n_sweep])[peak_f]), spec.str_unit, "   EwEw:", '{:+.2e}'.format(np.ravel(spec.EwEw   [n_sweep])[peak_f]), spec.str_unit)
if (spec.complex[0]>0):
    if   data.U_selected[n_time1//2] == 1 and data.V_selected[n_time1//2] == 1:  print("  EuEv_re:", '{:+.2e}'.format(np.ravel(spec.EuEv_re[n_sweep])[peak_f]), spec.str_unit, "  EuEv_im:", '{:+.2e}'.format(np.ravel(spec.EuEv_im[n_sweep])[peak_f]), spec.str_unit, "  PHAuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAuv[n_sweep])[peak_f]), "  COHuv:", '{:+4.2f}'.format(np.ravel(spec.E_COHuv[n_sweep])[peak_f]), "  DoPuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoPuv[n_sweep])[peak_f]), "  DoLuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoLuv[n_sweep])[peak_f]), "  DoCuv:", '{:+4.2f}'.format(np.ravel(spec.E_DoCuv[n_sweep])[peak_f]), "  ANGuv:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGuv[n_sweep])[peak_f]))
    if   data.V_selected[n_time1//2] == 1 and data.W_selected[n_time1//2] == 1:  print("  EvEw_re:", '{:+.2e}'.format(np.ravel(spec.EvEw_re[n_sweep])[peak_f]), spec.str_unit, "  EvEw_im:", '{:+.2e}'.format(np.ravel(spec.EvEw_im[n_sweep])[peak_f]), spec.str_unit, "  PHAvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAvw[n_sweep])[peak_f]), "  COHvw:", '{:+4.2f}'.format(np.ravel(spec.E_COHvw[n_sweep])[peak_f]), "  DoPvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoPvw[n_sweep])[peak_f]), "  DoLvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoLvw[n_sweep])[peak_f]), "  DoCvw:", '{:+4.2f}'.format(np.ravel(spec.E_DoCvw[n_sweep])[peak_f]), "  ANGvw:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGvw[n_sweep])[peak_f]))
    if   data.W_selected[n_time1//2] == 1 and data.U_selected[n_time1//2] == 1:  print("  EwEu_re:", '{:+.2e}'.format(np.ravel(spec.EwEu_re[n_sweep])[peak_f]), spec.str_unit, "  EwEu_im:", '{:+.2e}'.format(np.ravel(spec.EwEu_im[n_sweep])[peak_f]), spec.str_unit, "  PHAwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_PHAwu[n_sweep])[peak_f]), "  COHqu:", '{:+4.2f}'.format(np.ravel(spec.E_COHwu[n_sweep])[peak_f]), "  DoPwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoPwu[n_sweep])[peak_f]), "  DoLwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoLwu[n_sweep])[peak_f]), "  DoCwu:", '{:+4.2f}'.format(np.ravel(spec.E_DoCwu[n_sweep])[peak_f]), "  ANGwu:", '{:+5.1f} deg'.format(np.ravel(spec.E_ANGwu[n_sweep])[peak_f]))
    if   data.U_selected[n_time1//2] == 1 and data.V_selected[n_time1//2] == 1:  print('Stokes-UV: {:+.2e}'.format(np.ravel(spec.E_Iuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Quv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uuv[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vuv[n_sweep])[peak_f]))
    if   data.V_selected[n_time1//2] == 1 and data.W_selected[n_time1//2] == 1:  print('Stokes-VW: {:+.2e}'.format(np.ravel(spec.E_Ivw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uvw[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vvw[n_sweep])[peak_f]))
    if   data.W_selected[n_time1//2] == 1 and data.U_selected[n_time1//2] == 1:  print('Stokes-WU: {:+.2e}'.format(np.ravel(spec.E_Iwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Qwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Uwu[n_sweep])[peak_f]),   '{:+.2e}'.format(np.ravel(spec.E_Vwu[n_sweep])[peak_f]))

In [ ]:
n_sweep1 = 0;  n_sweep2 = n_time0//2;  n_sweep3 = n_time0-1
print(f"[First peak - {n_sweep1}]")
if data.U_selected[n_sweep1] == 1:  peak_E = np.ravel(spec.EuEu[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.V_selected[n_sweep1] == 1:  peak_E = np.ravel(spec.EvEv[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.W_selected[n_sweep1] == 1:  peak_E = np.ravel(spec.EwEw[n_sweep1]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Mid   peak - {n_sweep2}]")
if data.U_selected[n_sweep2] == 1:  peak_E = np.ravel(spec.EuEu[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.V_selected[n_sweep2] == 1:  peak_E = np.ravel(spec.EvEv[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.W_selected[n_sweep2] == 1:  peak_E = np.ravel(spec.EwEw[n_sweep2]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
print(f"[Last  peak - {n_sweep3}]")
if data.U_selected[n_sweep3] == 1:  peak_E = np.ravel(spec.EuEu[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EuEu:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.V_selected[n_sweep3] == 1:  peak_E = np.ravel(spec.EvEv[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EvEv:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))
if data.W_selected[n_sweep3] == 1:  peak_E = np.ravel(spec.EwEw[n_sweep3]); peak_f = np.nanargmax(peak_E); print("EwEw:", '{:.2e}'.format(peak_E[peak_f]), spec.str_unit, '{:.2e}'.format(peak_E[peak_f]**.5), '{:.1f} kHz'.format(freq_1d[peak_f]))

### Spectrum plot

In [ ]:
fig = plt.figure(figsize=(16, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(9, 1, 1);  ax2 = fig.add_subplot(9, 1, 2);  ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4);  ax5 = fig.add_subplot(9, 1, 5)
    ax6 = fig.add_subplot(9, 1, 6);  ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)
else:
    ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2);  ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)

ax1.plot(np.ravel(spec.EuEu),           '-r', linewidth=1.2, label='uu')
ax1.plot(np.ravel(spec.EvEv),           '-g', linewidth=0.8, label='vv')
ax1.plot(np.ravel(spec.EwEw),           '-b', linewidth=0.5, label='ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency),      '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*10),   '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width*10),  ':b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.cal_signal*10),  '-k', label='CAL')
ax8.plot(np.ravel(data.complex*10+4),   '_r', label='comp')
ax9.plot(np.ravel(data.epoch),          '.')
if comp_mode > 0:    # Matrix
    ax2.plot(np.ravel(spec.EuEv_re), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(spec.EvEw_re), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(spec.EwEu_re), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(spec.EuEv_im), ':r', linewidth=1.2, label='uv_im')
    ax2.plot(np.ravel(spec.EvEw_im), ':g', linewidth=1.0, label='vw_im')
    ax2.plot(np.ravel(spec.EwEu_im), ':b', linewidth=0.8, label='wu_im')
    ax3.plot(np.ravel(spec.E_DoPuv), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(spec.E_DoPvw), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(spec.E_DoPwu), '-b', linewidth=0.6, label='wu')
    ax4.plot(np.ravel(spec.E_DoLuv), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(spec.E_DoLvw), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(spec.E_DoLwu), '-b', linewidth=0.6, label='wu')
    ax5.plot(np.ravel(spec.E_DoCuv), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(spec.E_DoCvw), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(spec.E_DoCwu), '-b', linewidth=0.6, label='wu')
    ax6.plot(np.ravel(spec.E_ANGuv), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(spec.E_ANGvw), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(spec.E_ANGwu), '-b', linewidth=0.6, label='wu')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:   # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Lin Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.'); 
ax7.set_ylabel('frequency [kHz]');    ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time');  ax9.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR1-Rich (SID-21)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);     ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.EuEu)) -.5]; print(xlim); ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))-.5]; print(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];                ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
data.epoch

In [ ]:
n_time = n_time1//2
fig = plt.figure(figsize=(16, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(7, 1, 1);  ax2 = fig.add_subplot(7, 1, 2);  ax3 = fig.add_subplot(7, 1, 3);  ax4 = fig.add_subplot(7, 1, 4)
    ax5 = fig.add_subplot(7, 1, 5);  ax6 = fig.add_subplot(7, 1, 6);  ax7 = fig.add_subplot(7, 1, 7)
else:
    ax1 = fig.add_subplot(2, 1, 1);  ax7 = fig.add_subplot(2, 1, 2)

ax1.plot(spec.EuEu[n_time],          '-r', linewidth=1.2, label='uu')
ax1.plot(spec.EvEv[n_time],          '-g', linewidth=0.8, label='vv')
ax1.plot(spec.EwEw[n_time],          '-b', linewidth=0.5, label='ww')
ax1.set_yscale('log')
ax7.plot(data.frequency[n_time],     '-r', linewidth=0.6, label='frequency')
ax7.plot(data.freq_step[n_time]*10,  '-g', linewidth=0.8, label='step*10')
ax7.plot(data.freq_width[n_time]*10, ':b', linewidth=1.0, label='width*10')
if comp_mode > 0:    # Matrix
    ax2.plot(spec.EuEv_re[n_time], '-r', linewidth=1.0, label='uv_re')
    ax2.plot(spec.EvEw_re[n_time], '-g', linewidth=0.8, label='vw_re')
    ax2.plot(spec.EwEu_re[n_time], '-b', linewidth=0.6, label='wu_re')
    ax2.plot(spec.EuEv_im[n_time], ':r', linewidth=1.2, label='uv_im')
    ax2.plot(spec.EvEw_im[n_time], ':g', linewidth=1.0, label='vw_im')
    ax2.plot(spec.EwEu_im[n_time], ':b', linewidth=0.8, label='wu_im')
    ax3.plot(spec.E_DoPuv[n_time], '-r', linewidth=1.0, label='uv')
    ax3.plot(spec.E_DoPvw[n_time], '-g', linewidth=0.8, label='vw')
    ax3.plot(spec.E_DoPwu[n_time], '-b', linewidth=0.6, label='wu')
    ax4.plot(spec.E_DoLuv[n_time], '-r', linewidth=1.0, label='uv')
    ax4.plot(spec.E_DoLvw[n_time], '-g', linewidth=0.8, label='vw')
    ax4.plot(spec.E_DoLwu[n_time], '-b', linewidth=0.6, label='wu')
    ax5.plot(spec.E_DoCuv[n_time], '-r', linewidth=1.0, label='uv')
    ax5.plot(spec.E_DoCvw[n_time], '-g', linewidth=0.8, label='vw')
    ax5.plot(spec.E_DoCwu[n_time], '-b', linewidth=0.6, label='wu')
    ax6.plot(spec.E_ANGuv[n_time], '-r', linewidth=1.0, label='uv')
    ax6.plot(spec.E_ANGvw[n_time], '-g', linewidth=0.8, label='vw')
    ax6.plot(spec.E_ANGwu[n_time], '-b', linewidth=0.6, label='wu')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:    # Matrix
    ax2.set_ylabel('Real & Img');     ax3.set_ylabel('Deg. Pol.');  ax4.set_ylabel('Deg. Linear Pol.')
    ax5.set_ylabel('Deg. Cir Pol.');  ax6.set_ylabel('Lin Deg.')
ax7.set_ylabel('frequency [kHz]');    ax7.set_xlabel(data.epoch[n_time].strftime('%Y/%m/%d %R:%S') + '  [in ' + str_date + ']')
#
title_label = '[JUICE/RPWI HF PSSR1-Rich (SID-21)]\n' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);      ax7.legend(loc='upper right', fontsize=8)
if comp_mode > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);  ax6.legend(loc='upper right', fontsize=8)

xlim=[0, n_freq0];  ax1.set_xlim(xlim); ax7.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim)
# ylim=[10**p_raw_min, 10**p_raw_max]; # ax1.set_ylim(ylim)
if comp_mode > 0: # Matrix
    # ylim=[-10**p_max, 10**p_max];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0];  ax7.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw2.png'
    fig.savefig(png_fname)

In [ ]:
print("Complex:", data.complex)
print("CAL_sig:", data.cal_signal)

### FT Plot data

In [ ]:
# Normal spectrum plots
spec.EE    = spec.EuEu + spec.EvEv + spec.EwEw
EuEu_2d    = spec.EuEu.transpose();     EuEu_med    = np.nanmedian(spec.EuEu,    axis=0)
EvEv_2d    = spec.EvEv.transpose();     EvEv_med    = np.nanmedian(spec.EvEv,    axis=0)
EwEw_2d    = spec.EwEw.transpose();     EwEw_med    = np.nanmedian(spec.EwEw,    axis=0)
EE_2d      = spec.EE.transpose();       EE_med      = np.nanmedian(spec.EE,      axis=0)
E_DoPuv_2d = spec.E_DoLuv.transpose();  E_DoPuv_med = np.nanmedian(spec.E_DoPuv, axis=0)
E_DoPvw_2d = spec.E_DoLvw.transpose();  E_DoPvw_med = np.nanmedian(spec.E_DoPvw, axis=0)
E_DoPwu_2d = spec.E_DoLwu.transpose();  E_DoPwu_med = np.nanmedian(spec.E_DoPwu, axis=0)
E_DoLuv_2d = spec.E_DoLuv.transpose();  E_DoLuv_med = np.nanmedian(spec.E_DoLuv, axis=0)
E_DoLvw_2d = spec.E_DoLvw.transpose();  E_DoLvw_med = np.nanmedian(spec.E_DoLvw, axis=0)
E_DoLwu_2d = spec.E_DoLwu.transpose();  E_DoLwu_med = np.nanmedian(spec.E_DoLwu, axis=0)
E_DoCuv_2d = spec.E_DoCuv.transpose();  E_DoCuv_med = np.nanmedian(spec.E_DoCuv, axis=0)
E_DoCvw_2d = spec.E_DoCvw.transpose();  E_DoCvw_med = np.nanmedian(spec.E_DoCvw, axis=0)
E_DoCwu_2d = spec.E_DoCwu.transpose();  E_DoCwu_med = np.nanmedian(spec.E_DoCwu, axis=0)
E_ANGuv_2d = spec.E_ANGuv.transpose();  E_ANGuv_med = np.nanmedian(spec.E_ANGuv, axis=0)
E_ANGvw_2d = spec.E_ANGvw.transpose();  E_ANGvw_med = np.nanmedian(spec.E_ANGvw, axis=0)
E_ANGwu_2d = spec.E_ANGwu.transpose();  E_ANGwu_med = np.nanmedian(spec.E_ANGwu, axis=0)
E_COHuv_2d = spec.E_ANGwu.transpose();  E_COHuv_med = np.nanmedian(spec.E_COHuv, axis=0)
E_COHvw_2d = spec.E_ANGwu.transpose();  E_COHvw_med = np.nanmedian(spec.E_COHvw, axis=0)
E_COHwu_2d = spec.E_ANGwu.transpose();  E_COHwu_med = np.nanmedian(spec.E_COHwu, axis=0)
E_PHAuv_2d = spec.E_ANGwu.transpose();  E_PHAuv_med = np.nanmedian(spec.E_PHAuv, axis=0)
E_PHAvw_2d = spec.E_ANGwu.transpose();  E_PHAvw_med = np.nanmedian(spec.E_PHAvw, axis=0)
E_PHAwu_2d = spec.E_ANGwu.transpose();  E_PHAwu_med = np.nanmedian(spec.E_PHAwu, axis=0)

f_min0 = freq_1d[0];  f_max0 = freq_1d[N_frequency-1]
print(spec.str_unit, "   f_MAX/min:", f_max0, f_min0, "   num of f, step, window:", N_frequency, data.freq_step[0][0], data.freq_width[0][0])

In [ ]:
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
p_min = p_min0;  p_max = p_max0
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(16, 11))
if comp_mode > 0:    # Matrix
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)
else:
    ax1 = fig.add_subplot(1, 1, 1)

ax1.plot(freq_1d, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], '-r',  linewidth=1.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], '-g',  linewidth=0.8, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], '-b',  linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
#
if comp_mode > 0:    # Matrix
    ax2.plot(freq_1d, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax2.plot(freq_1d, spec.E_DoLvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax2.plot(freq_1d, spec.E_DoLwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax2.plot(freq_1d, spec.E_DoLuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax2.plot(freq_1d, spec.E_DoLvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax2.plot(freq_1d, spec.E_DoLwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax2.plot(freq_1d, spec.E_DoLuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax2.plot(freq_1d, spec.E_DoLvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax2.plot(freq_1d, spec.E_DoLwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax3.plot(freq_1d, spec.E_DoCuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax3.plot(freq_1d, spec.E_DoCvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax3.plot(freq_1d, spec.E_DoCwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax3.plot(freq_1d, spec.E_DoCuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax3.plot(freq_1d, spec.E_DoCvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax3.plot(freq_1d, spec.E_DoCwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax3.plot(freq_1d, spec.E_DoCuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax3.plot(freq_1d, spec.E_DoCvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax3.plot(freq_1d, spec.E_DoCwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax4.plot(freq_1d, spec.E_ANGuv[n0], ':r',  linewidth=1.0, label='uv init')
    ax4.plot(freq_1d, spec.E_ANGvw[n0], ':g',  linewidth=0.8, label='vw init')
    ax4.plot(freq_1d, spec.E_ANGwu[n0], ':b',  linewidth=0.6, label='wu init')
    ax4.plot(freq_1d, spec.E_ANGuv[n1], '-.r', linewidth=1.2, label='uv half')
    ax4.plot(freq_1d, spec.E_ANGvw[n1], '-.g', linewidth=1.0, label='vw half')
    ax4.plot(freq_1d, spec.E_ANGwu[n1], '-.b', linewidth=0.8, label='wu half')
    ax4.plot(freq_1d, spec.E_ANGuv[n2], '-r',  linewidth=1.2, label='uv end')
    ax4.plot(freq_1d, spec.E_ANGvw[n2], '-g',  linewidth=1.0, label='vw end')
    ax4.plot(freq_1d, spec.E_ANGwu[n2], '-b',  linewidth=0.8, label='wu end')
    ax5.plot(freq_1d, spec.E_DoLuv[n0], ':r',  linewidth=1.0, label='uv init')
    if f_mode == 1:
        ax2.set_xscale('log');  ax3.set_xscale('log');  ax4.set_xscale('log')

ax1.set_ylabel(spec.str_unit)
if comp_mode > 0:    # Matrix
    ax4.set_xlabel('frequency [Hz]');  ax2.set_ylabel('Deg. Linear Pol.');  ax3.set_ylabel('Deg. Circular Pol.');  ax4.set_ylabel('Linear Deg.')    
else:
    ax1.set_xlabel('frequency [Hz]')

date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)
if comp_mode > 0:
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); ax4.legend(loc='upper right', fontsize=8) 

xlim=[f_min, f_max];  ax1.set_xlim(xlim)
if comp_mode > 0:    # Matrix
    ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; ax1.set_ylim(ylim)
if comp_mode > 0:    # Matrix
    ylim=[-0.05, 1.05]; ax2.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax3.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax4.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

In [ ]:
p_min = p_min0;  p_max = p_max0
n0 = 0;  n1 = n_time0//2;   n2 = n_time0-1
f_min = f_min0;  f_max = f_max0

fig = plt.figure(figsize=(16, 11));  ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(freq_1d, EuEu_med,      '-r',  linewidth=2.0, label='EuEu_med')
ax1.plot(freq_1d, EvEv_med,      '-g',  linewidth=2.0, label='EvEv_med')
ax1.plot(freq_1d, EwEw_med,      '-b',  linewidth=2.0, label='EwEw_med')
ax1.plot(freq_1d, spec.EuEu[n0], ':r',  linewidth=1.0, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], ':g',  linewidth=0.8, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], ':b',  linewidth=0.6, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], '_r',  linewidth=1.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], '_g',  linewidth=0.8, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], '_b',  linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')

ax1.set_ylabel(spec.str_unit); ax1.set_xlabel('frequency [Hz]')
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)

xlim=[f_min, f_max]
ax1.set_xlim(xlim)
ylim=[10**p_min, 10**p_max]; 
ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode > 0:
    png_fname = work_dir+data_name+'_spec2.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec2-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
p_min = p_min0;  p_max = p_max0
# p_min = -14.5;     p_max = -13.5
f_min = f_min0;  f_max = f_max0
# f_min = 300;     f_max = 3000
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[16,11])
if time_mode == 1:
    ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)
    ax4.set_xlabel(str_date)
else:
    ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)
    ax3.set_xlabel(str_date)
ax1.set_ylim(f_min, f_max);   ax2.set_ylim(f_min, f_max);   ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');    ax2.set_yscale('log');        ax3.set_yscale('log')
ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]'); ax3.set_ylabel('EwEw [kHz]')
ax1.set_title("[" + data_name + "]")

if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(spec.str_unit)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(spec.str_unit)

print( "(", num_1d[0],")", Epoch_1d[0], "-", "(", num_1d[-1],")", Epoch_1d[-1] )
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    if t_min0 == 0:
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    else:
        xlim=[t_min0, t_max0]
    #E_min = '2024-08-21 04:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    #E_max = '2024-08-21 19:00:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    print("==>", xlim)
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.subplots_adjust(hspace=0.02);  plt.show()
if dump_mode > 0:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
# Z-range
p_min = p_min0;      p_max = p_max0

fig2d = plt.figure(figsize=[16,11], dpi=400)
ax1 = fig2d.add_subplot(2, 1, 1)
ax2 = fig2d.add_subplot(2, 1, 2)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log')

# X-axis
ax1.set_title("[" + data_name + "]")
if   data.U_selected[0]==1 and data.V_selected[0]==1:
    ax1.set_ylabel('EuEu [kHz]'); ax2.set_ylabel('EvEv [kHz]')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
elif data.V_selected[0]==1 and data.W_selected[0]==1:
    ax1.set_ylabel('EvEv [kHz]'); ax2.set_ylabel('EwEw [kHz]')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
else:
    ax1.set_ylabel('EvEv [kHz]'); ax2.set_ylabel('EuEu [kHz]')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
ax2.set_xlabel(str_date)
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)

plt.subplots_adjust(hspace=0.03)
plt.show()
if dump_mode > 0:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

## Ang-T

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(3, 1, 1);  ax2 = fig2d.add_subplot(3, 1, 2);  ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')

# X-axis
if   data.U_selected[0]==1 and data.V_selected[0]==1:
    ax1.set_title("[" + data_name + "]\n" + 'Deg. Lin. uv'); ax2.set_title('Deg. Cir. uv');     ax3.set_title('Ang. uv');
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
elif data.V_selected[0]==1 and data.W_selected[0]==1:
    ax1.set_title("[" + data_name + "]\n" + 'Deg. Lin. vw'); ax2.set_title('Deg. Cir. vw');     ax3.set_title('Ang. vw');  
    p2 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p5 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p8 = ax3.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
else:
    ax1.set_title("[" + data_name + "]\n" + 'Deg. Lin. wu'); ax2.set_title('Deg. Cir. wu');     ax3.set_title('Ang. wu')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='hsv')
ax3.set_xlabel(str_date)
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")

plt.subplots_adjust(hspace=0)
plt.show()
png_fname = work_dir+data_name+'_AngT.png'
if dump_mode == 1:
    if f_mode == 1:
        png_fname = work_dir+data_name+'_AngT-log.png'
    fig2d.savefig(png_fname)

# CLEANED data

In [ ]:
EuEu_c = juice_math.clean_rfi(np.ravel(spec.EuEu), 17)
EvEv_c = juice_math.clean_rfi(np.ravel(spec.EvEv), 17)
EwEw_c = juice_math.clean_rfi(np.ravel(spec.EwEw), 17)
EuEu_c = EuEu_c.reshape(n_time1, n_freq1)
EvEv_c = EvEv_c.reshape(n_time1, n_freq1)
EwEw_c = EwEw_c.reshape(n_time1, n_freq1)
EuEu_c_2d = EuEu_c.transpose()
EvEv_c_2d = EvEv_c.transpose()
EwEw_c_2d = EwEw_c.transpose()

## CLEANED - Spec

In [ ]:
# Sweep_num
n0 = 0
n1 = np.int16(n_time0/2)
n2 = -1
# X-range
f_min0 = f_min;  f_max0 = f_max
# Y-range
p_min0 = p_min;  p_max0 = p_max

fig = plt.figure(figsize=(16, 11))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

ax1.plot(freq_1d, spec.EuEu[n0], ':r', linewidth=1.0, label='uu init')
ax1.plot(freq_1d, spec.EvEv[n0], ':g', linewidth=0.8, label='vv init')
ax1.plot(freq_1d, spec.EwEw[n0], ':b', linewidth=0.6, label='ww init')
ax1.plot(freq_1d, spec.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq_1d, spec.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq_1d, spec.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq_1d, spec.EuEu[n2], '-r', linewidth=1.0, label='uu end')
ax1.plot(freq_1d, spec.EvEv[n2], '-g', linewidth=0.8, label='vv end')
ax1.plot(freq_1d, spec.EwEw[n2], '-b', linewidth=0.6, label='ww end')
ax2.plot(freq_1d, EuEu_c[n0], ':r', linewidth=1.0, label='uu init cleaned')
ax2.plot(freq_1d, EvEv_c[n0], ':g', linewidth=0.8, label='vv init cleaned')
ax2.plot(freq_1d, EwEw_c[n0], ':b', linewidth=0.6, label='ww init cleaned')
ax2.plot(freq_1d, EuEu_c[n1], '-.r', linewidth=1.0, label='uu half cleaned')
ax2.plot(freq_1d, EvEv_c[n1], '-.g', linewidth=0.8, label='vv half cleaned')
ax2.plot(freq_1d, EwEw_c[n1], '-.b', linewidth=0.6, label='ww half cleaned')
ax2.plot(freq_1d, EuEu_c[n2], '-r', linewidth=1.0, label='uu end cleaned')
ax2.plot(freq_1d, EvEv_c[n2], '-g', linewidth=0.8, label='vv end cleaned')
ax2.plot(freq_1d, EwEw_c[n2], '-b', linewidth=0.6, label='ww end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log')

# Label
ax1.set_ylabel(spec.str_unit);  ax2.set_ylabel(spec.str_unit)
ax2.set_xlabel('frequency [Hz]')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]\n" + date1 + "  -  " + date2 + "  -  " + date3;  ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min0, f_max0];         ax1.set_xlim(xlim); ax2.set_xlim(xlim);
# xlim=[1500, 1700];         ax1.set_xlim(xlim); ax2.set_xlim(xlim);
# range: Y-axis
# ylim=[10**p_min0, 10**p_max0]; ax1.set_ylim(ylim); ax2.set_ylim(ylim)

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log-c.png'
    # fig2d.savefig(png_fname)

## CLEANED -- FT

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[16,11])
ax1 = fig2d.add_subplot(2, 1, 1)
ax2 = fig2d.add_subplot(2, 1, 2)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]')

# X-axis
if   data.U_selected[0]==1 and data.V_selected[0]==1:
    ax1.set_title("[" + data_name + "]\n" + 'EuEu cleaned'); ax2.set_title('EvEv cleaned')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
elif data.V_selected[0]==1 and data.W_selected[0]==1:
    ax1.set_title("[" + data_name + "]\n" + 'EvEv cleaned'); ax2.set_title('EwEw cleaned')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
else:
    ax2.set_title("[" + data_name + "]\n" + 'EwEw cleaned'); ax3.set_title('EuEu cleaned')
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
ax2.set_xlabel(str_date)
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(spec.str_unit)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(spec.str_unit)

plt.subplots_adjust(hspace=0.03)
plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log-c.png'
    # fig2d.savefig(png_fname)